In [2]:
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import DataLoader
from model.model import Backbone, MobileFaceNet, Am_softmax, Arcface, PoseArcFace
import os.path as osp
import os, shutil
from utils.utils import separate_bn_paras
from easydict import EasyDict as edict
from pathlib import Path
from torchvision.datasets import ImageFolder
from sklearn.metrics import pairwise_distances, roc_curve, auc
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [10]:
retrained_model = Backbone(50, drop_ratio=0.6, mode='ir_se')
retrained_model.load_state_dict(torch.load('work_space/save/model_final_droneface.pth'))

<All keys matched successfully>

In [ ]:
folder_path = 'data/photos_all_faces/'
files = os.listdir(folder_path)

groups = {}

# Group the files by their first letter
for file_name in files:
    first_letter = file_name[0].upper()

    if first_letter not in groups:
        groups[first_letter] = []

    groups[first_letter].append(file_name)

# Create a new folder for each group and move the files
for first_letter, files in groups.items():
    new_folder_path = os.path.join(folder_path, first_letter)
    os.makedirs(new_folder_path, exist_ok=True)

    for file_name in files:
        old_file_path = os.path.join(folder_path, file_name)
        new_file_path = os.path.join(new_folder_path, file_name)

        shutil.move(old_file_path, new_file_path)

    print(f"Moved {len(files)} files to folder {first_letter}")


In [38]:
def get_config(training=True):
    conf = edict()
    conf.data_path = Path('data/photos_all_faces/')
    conf.work_path = Path('work_space/')
    conf.model_path = conf.work_path/'last_chechpoints'
    conf.save_path = conf.work_path/'save'
    conf.input_size = [112, 112]
    conf.embedding_size = 512
    conf.use_mobilefacenet = False
    conf.net_depth = 50
    conf.drop_ratio = 0.6
    conf.net_mode = 'ir_se'
    conf.class_num = 8
    conf.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    conf.test_transform = transforms.Compose([
        transforms.Resize((112, 112)), 
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    conf.batch_size = 32

    if training:
        conf.log_path = conf.work_path/'retrain'
        conf.save_path = conf.work_path/'save'
        conf.lr = 1e-3
        conf.momentum = 0.9
        conf.pin_memory = True
        conf.num_workers = 3
        conf.ce_loss = nn.CrossEntropyLoss()
    else:
        pass 
    return conf

In [39]:
conf = get_config()

Create model, head and optimizer then load weights

In [40]:
device = torch.device('cuda')

if conf.use_mobilefacenet:
    model = MobileFaceNet(embedding_size=conf.net_depth).to(device)
else:
    model = Backbone(num_layers=conf.net_depth, drop_ratio=conf.drop_ratio, mode=conf.net_mode).to(device)

paras_only_bn, paras_wo_bn = separate_bn_paras(model)

head = Arcface(embedding_size=conf.embedding_size, classnum=conf.class_num).to(device)

if conf.use_mobilefacenet:
    optimizer = optim.SGD([
        {'params': paras_wo_bn[:-1], 'weight_decay': 4e-5},
        {'params': [paras_wo_bn[-1]] + [head.kernel], 'weight_decay': 4e-4},
        {'params': paras_only_bn}
    ], lr=conf.lr, momentum=conf.momentum)
else:
    optimizer = optim.SGD([
        {'params': paras_wo_bn + [head.kernel], 'weight_decay': 5e-4},
        {'params': paras_only_bn}
    ], lr=conf.lr, momentum=conf.momentum)

# checkpoint_path = conf.model_path

if conf.use_mobilefacenet:
    model_path = 'work_space/last_checkpoints/model_mobilefacenet.pth'
    model.load_state_dict(torch.load(model_path))
else:
    model_path = 'work_space/last_checkpoints/model_ir_se50.pth'
    model.load_state_dict(torch.load(model_path))
# head_path = 'work_space/last_checkpoints/head_2023-03-08-07-50_accuracy:0.8991428571428571_step:1346430_None.pth'
# head.load_state_dict(torch.load(head_path))
# optimizer_path = 'work_space/last_checkpoints/optimizer_2023-03-08-07-50_accuracy:0.8991428571428571_step:1346430_None.pth'
# optimizer.load_state_dict(torch.load(optimizer_path))

In [41]:
train_dir = osp.join(conf.data_path, 'train')
test_dir = osp.join(conf.data_path, 'test')

train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

train_data = ImageFolder(
    root=train_dir,
    transform=train_transform,
    target_transform=None
)
test_data = ImageFolder(
    root=test_dir,
    transform=conf.test_transform,
    target_transform=None
)

train_loader = DataLoader(train_data, batch_size=16, shuffle=True, pin_memory=conf.pin_memory, num_workers=conf.num_workers)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False, pin_memory=conf.pin_memory, num_workers=conf.num_workers)



In [43]:


def train(conf, epochs, train_loader, model, optimizer, device):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0. 
        print(f'Epoch {epoch} started')
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            embeddings = model(imgs)
            thetas = head(embeddings, labels)
            loss = conf.ce_loss(thetas, labels)
            running_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            
            optimizer.step()
        print(running_loss/len(train_loader))
    return model 

trained_model = train(conf, 10, train_loader, model, optimizer, device)


Epoch 0 started
0.07955816212666235
Epoch 1 started
0.0950562267227704
Epoch 2 started
0.08993127986561271
Epoch 3 started
0.004169204059788397
Epoch 4 started
0.04273069532224194
Epoch 5 started
0.010708099017723344
Epoch 6 started
0.05907386103390898
Epoch 7 started
0.0420092335612425
Epoch 8 started
0.004678101877857408
Epoch 9 started
0.011995586384655999


In [44]:
# Create a dictionary that maps each class name to a class index
class_to_idx = test_data.class_to_idx

# Create a list of image paths and class indices
image_paths = []
class_indices = []
for target_class in class_to_idx.keys():
    target_class_path = os.path.join(test_dir, target_class)
    for image_filename in os.listdir(target_class_path):
        image_path = os.path.join(target_class_path, image_filename)
        image_paths.append(image_path)
        class_indices.append(class_to_idx[target_class])


In [45]:
# Create a list of image embeddings
embeddings = []
trained_model.eval()
with torch.inference_mode():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = trained_model(inputs)
        embeddings.append(outputs)

embeddings = torch.cat(embeddings).cpu()

In [47]:
distances = pairwise_distances(embeddings, metric='euclidean')

In [61]:
# Calculate the CMC curve
# matches = (np.argsort(distances, axis=1) == class_indices[:, np.newaxis]).astype(np.int32)
matches = (np.argsort(distances, axis=1) == np.array(class_indices)[:, np.newaxis]).astype(np.int32)
cmc = np.cumsum(np.any(matches, axis=1))

# Calculate the false positive rate and true positive rate for the ROC curve
n_classes = len(np.unique(class_indices))
n_imgs_per_class = len(class_indices) // n_classes
true_labels = np.zeros(len(embeddings))
for i, class_idx in enumerate(class_indices):
    true_labels[class_idx: class_idx + n_imgs_per_class] = 1
fpr, tpr, _ = roc_curve(true_labels, -embeddings[:, 0])
roc_auc = auc(fpr, tpr)
roc_auc

0.0033230094205703962

In [60]:
# Visualize the CMC curve
plt.plot(cmc / len(class_indices))
plt.title('CMC curve')
plt.xlabel('Rank')
plt.ylabel('Identification rate')
plt.show()

# Visualize the ROC curve
plt.plot(fpr, tpr)
plt.title('ROC curve (AUC = {:.2f})'.format(roc_auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show()